MOVIE RECOMMENDER SYSTEM

Importing the dependencies

In [25]:
import numpy as np
import pandas as pd
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

Data Collection and pre-Processing

In [26]:
#loading the data from the csv file to pandas dataframe
movies_data= pd.read_csv('movies.csv')

In [28]:
#printing the first 5 data
movies_data.head()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


In [5]:
#number of rows and coloumns
movies_data.shape

(4803, 24)

In [13]:
#selecting the relevant features for recommendation
selected_features=['genres','keywords','tagline','cast','director']
print(selected_features)

['genres', 'keywords', 'tagline', 'cast', 'director', 'homepage']


index                      0
budget                     0
genres                     0
homepage                3091
id                         0
keywords                   0
original_language          0
original_title             0
overview                   3
popularity                 0
production_companies       0
production_countries       0
release_date               1
revenue                    0
runtime                    2
spoken_languages           0
status                     0
tagline                    0
title                      0
vote_average               0
vote_count                 0
cast                       0
crew                       0
director                   0
dtype: int64

In [17]:
#repacing the missing values with null string
for feature in selected_features:
    movies_data[feature]= movies_data[feature].fillna('')


In [19]:
#combining all the 5 selectes features
combined_features = movies_data['genres']+movies_data['keywords']+movies_data['cast']+movies_data['tagline']+movies_data['director']
print(combined_features)

0       Action Adventure Fantasy Science Fictioncultur...
1       Adventure Fantasy Actionocean drug abuse exoti...
2       Action Adventure Crimespy based on novel secre...
3       Action Crime Drama Thrillerdc comics crime fig...
4       Action Adventure Science Fictionbased on novel...
                              ...                        
4798    Action Crime Thrillerunited states\u2013mexico...
4799    Comedy RomanceEdward Burns Kerry Bish\u00e9 Ma...
4800    Comedy Drama Romance TV Moviedate love at firs...
4801    Daniel Henney Eliza Coupe Bill Paxton Alan Ruc...
4802    Documentaryobsession camcorder crush dream gir...
Length: 4803, dtype: object


In [20]:
#converting the text data into the feature vertex
vectorizer=TfidfVectorizer()

In [22]:
feature_vectors= vectorizer.fit_transform(combined_features)
print(feature_vectors)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 111730 stored elements and shape (4803, 27793)>
  Coords	Values
  (0, 249)	0.0787885735193979
  (0, 449)	0.09132857870544613
  (0, 9006)	0.11836592141063045
  (0, 22021)	0.09998166261909516
  (0, 9381)	0.27456446600852247
  (0, 4480)	0.21855919626397677
  (0, 10295)	0.16575087043546832
  (0, 23106)	0.337556237272303
  (0, 26544)	0.13093417905590005
  (0, 4701)	0.2459238418151712
  (0, 22971)	0.27456446600852247
  (0, 27391)	0.23540668329185652
  (0, 27753)	0.1996131269037809
  (0, 21668)	0.2160572904494248
  (0, 22654)	0.2160572904494248
  (0, 26765)	0.19689340044337153
  (0, 23464)	0.15455418218914943
  (0, 14664)	0.22425804599456597
  (0, 16741)	0.1596538352412605
  (0, 20901)	0.27456446600852247
  (0, 24390)	0.07537942267634702
  (0, 27372)	0.12652446609697676
  (0, 18317)	0.08713254643393821
  (0, 18714)	0.2618907879051872
  (0, 13347)	0.116223484793762
  :	:
  (4801, 18945)	0.22049236993953694
  (4801, 11877)	0.29519313

Cosine Similarity